In [1]:
import sys

In [2]:
#Insert path to Separated Topologies repo
sys.path.insert(1, '/Users/hannahbaumann/Google Drive/sep_top/SeparatedTopologies/')

In [3]:
import make_septop as ms
import combine_coordinates as ac
import boresch_restraints as br
import distance_solvent as ds
import os
import parmed as pmd

# Create input files for the complex

## Defining input file names

In [13]:
#path to data folder
path = 'complex'
#ligand folders: name of this folder is considered the ligand name
ligand_A = 'lig_13'
ligand_B = 'lig_16'
#directories
compound_A = '%s/%s'%(path, ligand_A)
compound_B = '%s/%s'%(path, ligand_B)
#name of mol2 files ligand
mol2_A = '%s/%s.mol2'%(compound_A, ligand_A)
mol2_B = '%s/%s.mol2'%(compound_B, ligand_B)
#name of complex files
pdb_A = '%s/complex.pdb'%(compound_A)
pdb_B = '%s/complex.pdb'%(compound_B)
gro_A = '%s/complex.gro'%(compound_A)
#Input of an equilibrium simulation for Boresch selection is optional, 
#but recommended for selection of suitable atoms for restraints
#Not including trajectory here since file too large, using last frame equilibrium run instead
traj_A = '%s/complex.gro'%(compound_A)
traj_B = '%s/complex.gro'%(compound_B)
top_A = '%s/complex.top'%(compound_A)
top_B = '%s/complex.top'%(compound_B)

In [5]:
#scaling factor gamma for eps-HREX
gamma = 0.5

In [6]:
#Three letter code for ligands
lig = 'LIG'

In [7]:
#Create a folder for the edge
edge_A_B = '%s/edge_%s_%s'%(path, ligand_A, ligand_B)
if not os.path.isdir(edge_A_B):os.mkdir(edge_A_B)

## Generate SepTop coordinate file

In [8]:
#define some names of intermediate files of aligned complexes
fit_B = '%s/complex_fit.pdb'%compound_B
gro_B = '%s/complex_fit.gro'%compound_B
#define name of output file (SepTop coordinate file)
complex = '%s/complex.gro'%edge_A_B

In [9]:
# Align complexed to be able to insert ligand B into complex of ligand A
# *** This part requires Openeye Spruce license. If not available, please use different alignment tool ***
ac.align_complexes(pdb_A, pdb_B, fit_B)

In [10]:
# Convert .pdb to .gro
ac.pdb2gro(fit_B, gro_B)

<Structure 63421 atoms; 19583 residues; 44315 bonds; PBC (orthogonal); NOT parametrized>


In [11]:
#Insert ligand B  coordinates into coordinate file or complex A
complex = ac.combine_ligands_gro(gro_A, gro_B, complex, ligand_A=lig, ligand_B=lig)

In [12]:
# Edit indices
complex = ac.edit_indices(complex,complex)

## Pick atoms for Boresch restraints and add that section

In [14]:
bore_A_on = '%s/boresch_restraints_A_on.itp'%edge_A_B
bore_B_off = '%s/boresch_restraints_B_off.itp'%edge_A_B
bore_A = '%s/boresch_restraints_A.itp'%edge_A_B
bore_B = '%s/boresch_restraints_B.itp'%edge_A_B
restrain_A, restrain_B, dG_A_off, dG_B_on = br.restrain_ligands(traj_A,traj_B,mol2_A,mol2_B, 
                                            bore_A_on, bore_B_off, bore_A, bore_B,
                                            ligand_atoms=None, protein_atoms=None, substructure=None,
                                            ligand=lig, top_A=gro_A, top_B=gro_B)

/Users/hannahbaumann/miniconda3/envs/orion/lib/python3.7/site-packages/mdtraj/core/trajectory.py:419: UserWarning: top= kwarg ignored since file contains topology information
  warnings.warn('top= kwarg ignored since file contains topology information')
/Users/hannahbaumann/miniconda3/envs/orion/lib/python3.7/site-packages/mdtraj/utils/validation.py:116: TypeCastPerformanceWarning: Casting unitcell_vectors dtype=float64 to <class 'numpy.float32'> 
  TypeCastPerformanceWarning)


[1.32, 50.49, 84.27, -89.4, -64.22, 131.45]
[1.51, 47.25, 124.09, -86.06, -8.43, -87.74]
[2933, 511, 493, 6053, 6054, 6059]
[2933, 511, 493, 6105, 6103, 6097]


## Generating the separated topology

### 1. Insert ligand B into the same molecule section as ligand A in complex A

In [15]:
top = '%s/complex.top'%edge_A_B
#Combine topology files
ms.combine_ligands_top(top_A, top_B, top, ligand=lig)

In [16]:
#Name top output files for two steps
step_1 = '%s/complex_1_scale.top'%edge_A_B
step_2 = '%s/complex_2_scale.top'%edge_A_B

### 2. Generate separated topology files for step1

In [17]:
# Generate separated topology file
ms.create_top(top, step_1, gamma, 'vdwq_scaled-vdw', 'dummy_scaled-vdwq', top_A, top_B, ligand=lig)
#Include Boresch .itp files into .top file 
br.include_itp_in_top(step_1, bore_B)
br.include_itp_in_top(step_1, bore_A_on)

### 3. Generate separated topology files for step2

In [18]:
# Generate separated topology file
ms.create_top(top, step_2, gamma, 'scaled-vdw_dummy', 'scaled-vdwq_vdwq', top_A, top_B, ligand=lig)
#Include Boresch .itp files into .top file 
br.include_itp_in_top(step_2, bore_B_off)
br.include_itp_in_top(step_2, bore_A)

# Create input files for the solvent

## Defining input file names

In [19]:
#path to data folder
path = 'solvent'

#ligand folders: name of this folder is considered the ligand name
ligand_A = 'lig_13'
ligand_B = 'lig_16'

#directories of ligand input files
compound_A = '%s/%s'%(path, ligand_A)
compound_B = '%s/%s'%(path, ligand_B)

#name of solvent .gro and .top files
gro_A = '%s/solvent.gro'%compound_A
gro_B = '%s/solvent.gro'%compound_B
top_A = '%s/solvent.top'%compound_A
top_B = '%s/solvent.top'%compound_B

#scaling factor gamma
gamma = 0.3

#Create new folder for each edge
edge_A_B = '%s/edge_%s_%s'%(path, ligand_A, ligand_B)
if not os.path.isdir(edge_A_B):os.mkdir(edge_A_B)

#Three letter code for ligands
lig = 'UNL'

## Generate SepTop coordinate file

In [20]:
solv_gro = '%s/solvent.gro'%edge_A_B
ac.combine_ligands_gro(gro_A, gro_B, solv_gro, ligand_A=lig, ligand_B=lig)
ac.edit_indices(solv_gro,solv_gro)

## Generating the separated topology

### 1. Insert ligand B into the same molecule section as ligand A in complex A

In [21]:
#Specify output file names
top = '%s/solvent.top'%edge_A_B
step_1 = '%s/step1.top'%edge_A_B
step_1_eq = '%s/step1_eq.top'%edge_A_B
step_2 = '%s/step2.top'%edge_A_B
step_2_eq = '%s/step2_eq.top'%edge_A_B

In [22]:
#Combine topology ligand B and topology A
ms.combine_ligands_top(top_A, top_B, top, ligand=lig)

### 2. Generate separated topology files for step1

In [23]:
#Create complex step 1
ms.create_top(top, step_1, gamma, 'vdwq_scaled-vdw', 'dummy_scaled-vdwq', top_A, top_B, ligand=lig)
ms.create_top(top, step_1_eq, gamma, 'vdwq_scaled-vdw', 'dummy_scaled-vdwq', top_A, top_B, ligand=lig)

'solvent/edge_lig_13_lig_16/step1_eq.top'

### 3. Generate separated topology files for step2

In [24]:
#Create complex step 2
ms.create_top(top, step_2, gamma, 'scaled-vdw_dummy', 'scaled-vdwq_vdwq', top_A, top_B, ligand=lig)
ms.create_top(top, step_2_eq, gamma, 'scaled-vdw_dummy', 'scaled-vdwq_vdwq', top_A, top_B, ligand=lig)

'solvent/edge_lig_13_lig_16/step2_eq.top'

### 4. Add .itp file with distance restraint

In [25]:
#Load in .top and .gro and save .gro again to fix residue numbers
gro = pmd.load_file(step_1, xyz=solv_gro)
gro.save(solv_gro, overwrite=True)

In [26]:
#define value for force constants
#equilibrate first with low force constant to avoid instabilities
fc_eq = 1.0
fc_prod = 1000.0

In [27]:
#restrain atoms closest to the COM of the two ligands using a single harmonic distance restraint
atoms_restrain,dist = ds.distance_restraint(solv_gro,lig)
ds.write_itp_restraints(atoms_restrain, dist, fc_eq,fc_eq, '%s/dist_restraint_eq.itp'%edge_A_B)
ds.write_itp_restraints(atoms_restrain, dist, fc_prod,fc_prod, '%s/dist_restraint.itp'%edge_A_B)

In [28]:
#Include .itp files in the topology files
br.include_itp_in_top(step_1, 'dist_restraint.itp')
br.include_itp_in_top(step_1_eq, 'dist_restraint_eq.itp')
br.include_itp_in_top(step_2, 'dist_restraint.itp')
br.include_itp_in_top(step_2_eq, 'dist_restraint_eq.itp')